In [44]:
import glob, re
import numpy as np
import pandas as pd
from sklearn import *
import datetime as dt
import gc
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
import pickle

In [45]:
train = pd.read_csv("../Input/train_proc2.csv")
test = pd.read_csv("../Input/test_proc2.csv")
submission = pd.read_csv("../Input/sample_submission.csv")
air_store = pd.read_csv("../Input/allstore_info_proc1.csv").rename(columns={'store_id':'air_store_id'})
date_info = pd.read_csv("../Input/date_info.csv").rename(columns={"calendar_date" : 'visit_date'})

In [46]:
submission['visit_date'] = submission['id'].map(lambda x: str(x).split('_')[2])
submission['air_store_id'] = submission['id'].map(lambda x: '_'.join(x.split('_')[:2]))
submission['visit_date'] = pd.to_datetime(submission['visit_date'])
submission['visitors'] = np.nan

In [47]:
train['visit_date'] = pd.to_datetime(train['visit_date'])
test['visit_date'] = pd.to_datetime(test['visit_date'])
test['visitors'] = -1

In [48]:
date_info['visit_date'] = pd.to_datetime(date_info['visit_date'])

In [49]:
date_info['date_seq_id'] = date_info['visit_date'].dt.date - date_info['visit_date'].dt.date.min() + dt.timedelta(1)
date_info['date_seq_id'] = date_info['date_seq_id']/dt.timedelta(1)

In [50]:
date_info['week_seq_id'] = ((date_info['date_seq_id']+4)/7).astype(np.int64)

In [51]:
#train = train[train['visit_date'] >= dt.date(2016,3,1)]

In [52]:
stores_to_drop = list(set(train['air_store_id']) - set(test['air_store_id']))
train = train.where(~(train['air_store_id'].isin(stores_to_drop)))
train = train.dropna(axis=0,subset=['air_store_id'])

In [53]:
all_data = pd.concat([train,test])

In [54]:
train_len = len(train)

In [55]:
del train; del test;
gc.collect();

In [56]:
all_data = pd.merge(all_data,air_store,how='left',on='air_store_id',)

In [57]:
all_data = pd.merge(all_data,date_info,how='left',on='visit_date')

In [58]:
all_data['visit_month'] = all_data['visit_date'].dt.month
all_data['visit_month'] = all_data['visit_date'].dt.year

In [59]:
# dont know why this would help ??
all_data['diff_max_lat'] = all_data['latitude'].max() - all_data['latitude']
all_data['diff_min_lat'] = all_data['latitude'].min() - all_data['latitude']
all_data['diff_max_long'] = all_data['longitude'].max() - all_data['longitude']
all_data['diff_min_long'] = all_data['longitude'].min() - all_data['longitude']
all_data['lat_plus_long'] = all_data['latitude'] + all_data['longitude']

In [60]:
stores_to_drop = ['air_b2d8bc9c88b85f96',
 'air_cf22e368c1a71d53',
 'air_229d7e508d9f1b5e',
 'air_d0a7bd3339c3d12a',
 'air_cb083b4789a8d3a2',
 'air_2703dcb33192b181',
 'air_0ead98dd07e7a82a',
 'air_d63cfa6d6ab78446']

In [61]:
all_data = all_data.where(~(all_data['air_store_id'].isin(stores_to_drop)))
all_data = all_data.dropna(axis=0,subset=['air_store_id'])

In [62]:
cols_to_encode = ([i for i,j in zip(all_data.dtypes.index,all_data.dtypes.values)
                 if j == 'object'])
#cols_to_encode.remove('visit_date')

In [63]:
lbl = preprocessing.LabelEncoder()
for i in cols_to_encode:
    all_data[i] = lbl.fit_transform(all_data[i])

In [64]:
all_data.shape

(282487, 107)

In [65]:
train = all_data[:train_len]

In [66]:
train.shape

(250468, 107)

In [67]:
del air_store;del date_info;del all_data; gc.collect();

In [68]:
train['visit_date'] = pd.to_datetime(train['visit_date'])

In [69]:
train['visit_date'] = train['visit_date'].dt.date

In [70]:
train.sort_values('visit_date').iloc[-1]['visit_date']-dt.timedelta(days=90) #38days validation set

datetime.date(2017, 1, 22)

In [71]:
test = train[train['visit_date'] > dt.date(2016, 12, 23)]

In [29]:
train = train[train['visit_date'] <= dt.date(2016, 12, 23)]

In [30]:
train = train.fillna(-1)
test = test.fillna(-1)

In [31]:
# %load prepareData.py
def prepareData(trainIn, testIn):
    stat1 = trainIn.groupby(["air_store_id","day_of_week"],as_index=False)['visitors'].sum().rename(columns={'visitors':'visitors_sum_store_day'})
    stat2 = trainIn.groupby(["air_store_id","day_of_week"],as_index=False)['visitors'].mean().rename(columns={'visitors':'visitors_mean_store_day'})
        
    stat3 = trainIn.groupby(["area_id","day_of_week"],as_index=False)['visitors'].sum().rename(columns={'visitors':'visitors_sum_area'})
    stat4 = trainIn.groupby(["area_id","day_of_week"],as_index=False)['visitors'].mean().rename(columns={'visitors':'visitors_mean_area'})

    stat5 = trainIn.groupby(["n200mt_cluster_id","day_of_week"],as_index=False)['visitors'].sum().rename(columns={'visitors':'visitors_sum_200mt'})
    stat6 = trainIn.groupby(["n200mt_cluster_id","day_of_week"],as_index=False)['visitors'].mean().rename(columns={'visitors':'visitors_mean_200mt'})

    stat7 = trainIn.groupby(["n400mt_cluster_id","day_of_week"],as_index=False)['visitors'].sum().rename(columns={'visitors':'visitors_sum_400mt'})
    stat8 = trainIn.groupby(["n400mt_cluster_id","day_of_week"],as_index=False)['visitors'].mean().rename(columns={'visitors':'visitors_mean_400mt'})
    
    stat9 = trainIn.groupby(["n1000mt_cluster_id","day_of_week"],as_index=False)['visitors'].sum().rename(columns={'visitors':'visitors_sum_1000mt'})
    stat10 = trainIn.groupby(["n1000mt_cluster_id","day_of_week"],as_index=False)['visitors'].mean().rename(columns={'visitors':'visitors_mean_1000mt'})
    
    stat11 = trainIn.groupby(["n200mt_cluster_id","genre_name"],as_index=False)['visitors'].sum().rename(columns={'visitors':'visitors_sum_genre_200mt'})
    stat12 = trainIn.groupby(["n200mt_cluster_id","genre_name"],as_index=False)['visitors'].mean().rename(columns={'visitors':'visitors_mean_genre_200mt'})
    
    stat13 = trainIn.groupby(["n400mt_cluster_id","genre_name"],as_index=False)['visitors'].sum().rename(columns={'visitors':'visitors_sum_genre_400mt'})
    stat14 = trainIn.groupby(["n400mt_cluster_id","genre_name"],as_index=False)['visitors'].mean().rename(columns={'visitors':'visitors_mean_genre_400mt'})
    
    stat15 = trainIn.groupby(["n1000mt_cluster_id","genre_name"],as_index=False)['visitors'].sum().rename(columns={'visitors':'visitors_sum_genre_1000mt'})
    stat16 = trainIn.groupby(["n1000mt_cluster_id","genre_name"],as_index=False)['visitors'].mean().rename(columns={'visitors':'visitors_mean_genre_1000mt'})
    #####################################
    stat17 = trainIn.groupby(["air_store_id","day_of_week",'week_seq_id'],as_index=False)['visitors'].sum().rename(columns={'visitors':'visitors_sum_store_day'})
    #stat18 = trainIn.groupby(["air_store_id","day_of_week",'week_seq_id'],as_index=False)['visitors'].mean().rename(columns={'visitors':'visitors_mean_store_day'})
    #stat17 = pd.merge(stat17, stat18, on=['air_store_id','day_of_week','week_seq_id'])
    #del stat18; gc.collect()
    
    stat_6wago_day = stat17.copy()
    stat_6wago_day.loc[:,'week_seq_id'] = np.nan
    stat_6wago_day.loc[:,'week_seq_id'] = stat17.loc[:,'week_seq_id'] + 6
    stat_6wago_day = stat_6wago_day.rename(columns={'visitors_sum_store_day' : 'visitors_6wks_wago'})
    
    stat_7wago_day = stat17.copy()
    stat_7wago_day.loc[:,'week_seq_id'] = np.nan
    stat_7wago_day.loc[:,'week_seq_id'] = stat17.loc[:,'week_seq_id'] + 7
    stat_7wago_day = stat_7wago_day.rename(columns={'visitors_sum_store_day' : 'visitors_7wks_wago'})
   
    stat_8wago_day = stat17.copy()
    stat_8wago_day.loc[:,'week_seq_id'] = np.nan
    stat_8wago_day.loc[:,'week_seq_id'] = stat17.loc[:,'week_seq_id'] + 8
    stat_8wago_day = stat_8wago_day.rename(columns={'visitors_sum_store_day' : 'visitors_8wks_wago'})

    stat_9wago_day = stat17.copy()
    stat_9wago_day.loc[:,'week_seq_id'] = np.nan
    stat_9wago_day.loc[:,'week_seq_id'] = stat17.loc[:,'week_seq_id'] + 9
    stat_9wago_day = stat_9wago_day.rename(columns={'visitors_sum_store_day' : 'visitors_9wks_wago'})
    
    stat_10wago_day = stat17.copy()
    stat_10wago_day.loc[:,'week_seq_id'] = np.nan
    stat_10wago_day.loc[:,'week_seq_id'] = stat17.loc[:,'week_seq_id'] + 10
    stat_10wago_day = stat_10wago_day.rename(columns={'visitors_sum_store_day' : 'visitors_10wks_wago'})
    
    #################################
    stat18 = trainIn.groupby(["air_store_id","week_seq_id"],as_index=False)['visitors'].sum().rename(columns={'visitors':'visitors_sum_store_week'})
    stat19 = trainIn.groupby(["air_store_id","week_seq_id"],as_index=False)['visitors'].mean().rename(columns={'visitors':'visitors_mean_store_week'})                  
    stat18 = pd.merge(stat18, stat19, on=['air_store_id','week_seq_id'])
    del stat19;
    
    stat_6wago = stat18.copy()
    stat_6wago.loc[:,'week_seq_id'] = np.nan
    stat_6wago.loc[:,'week_seq_id'] = stat18.loc[:,'week_seq_id'] + 6
    stat_6wago = stat_6wago.rename(columns={'visitors_sum_store_week' : 'visitors_sum_store_6wago'})
    stat_6wago = stat_6wago.rename(columns={'visitors_mean_store_week' : 'visitors_mean_store_6wago'})
    
    stat_7wago = stat18.copy()
    stat_7wago.loc[:,'week_seq_id'] = np.nan
    stat_7wago.loc[:,'week_seq_id'] = stat18.loc[:,'week_seq_id'] + 7
    stat_7wago = stat_7wago.rename(columns={'visitors_sum_store_week' : 'visitors_sum_store_7wago'})
    stat_7wago = stat_7wago.rename(columns={'visitors_mean_store_week' : 'visitors_mean_store_8wago'})
    
    stat_8wago = stat18.copy()
    stat_8wago.loc[:,'week_seq_id'] = np.nan
    stat_8wago.loc[:,'week_seq_id'] = stat18.loc[:,'week_seq_id'] + 8
    stat_8wago = stat_8wago.rename(columns={'visitors_sum_store_week' : 'visitors_sum_store_day_6wago'})
    stat_8wago = stat_8wago.rename(columns={'visitors_mean_store_week' : 'visitors_mean_store_day_6wago'}) 
    
    stat_9wago = stat18.copy()
    stat_9wago.loc[:,'week_seq_id'] = np.nan
    stat_9wago.loc[:,'week_seq_id'] = stat18.loc[:,'week_seq_id'] + 9
    stat_9wago = stat_9wago.rename(columns={'visitors_sum_store_week' : 'visitors_sum_store_9wago'})
    stat_9wago = stat_9wago.rename(columns={'visitors_mean_store_week' : 'visitors_mean_store_9wago'})
    
    stat_10wago = stat18.copy()
    stat_10wago.loc[:,'week_seq_id'] = np.nan
    stat_10wago.loc[:,'week_seq_id'] = stat18.loc[:,'week_seq_id'] + 10
    stat_10wago = stat_10wago.rename(columns={'visitors_sum_store_week' : 'visitors_sum_store_10wago'})
    stat_10wago = stat_10wago.rename(columns={'visitors_mean_store_week' : 'visitors_mean_store_10wago'}) 
    ##############################
    
    trainIn = pd.merge(trainIn, stat1, on = ["air_store_id", "day_of_week"], how='left')
    trainIn = pd.merge(trainIn, stat2, on = ["air_store_id", "day_of_week"], how='left')
    trainIn = pd.merge(trainIn, stat3, on = ["area_id", "day_of_week"], how='left')
    trainIn = pd.merge(trainIn, stat4, on = ["area_id", "day_of_week"], how='left')
    
    trainIn = pd.merge(trainIn, stat_6wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    trainIn = pd.merge(trainIn, stat_7wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    trainIn = pd.merge(trainIn, stat_8wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    trainIn = pd.merge(trainIn, stat_9wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    trainIn = pd.merge(trainIn, stat_10wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    
    trainIn = pd.merge(trainIn, stat_6wago, on = ["air_store_id", "week_seq_id"], how='left')
    trainIn = pd.merge(trainIn, stat_7wago, on = ["air_store_id", "week_seq_id"], how='left')
    trainIn = pd.merge(trainIn, stat_8wago, on = ["air_store_id", "week_seq_id"], how='left')
    trainIn = pd.merge(trainIn, stat_9wago, on = ["air_store_id", "week_seq_id"], how='left')
    trainIn = pd.merge(trainIn, stat_10wago, on = ["air_store_id", "week_seq_id"], how='left')
    
    trainIn = pd.merge(trainIn, stat5, on = ["n200mt_cluster_id", "day_of_week"], how='left')
    trainIn = pd.merge(trainIn, stat6, on = ["n200mt_cluster_id", "day_of_week"], how='left')
    trainIn = pd.merge(trainIn, stat7, on = ["n400mt_cluster_id", "day_of_week"], how='left')
    trainIn = pd.merge(trainIn, stat8, on = ["n400mt_cluster_id", "day_of_week"], how='left')
    trainIn = pd.merge(trainIn, stat9, on = ["n1000mt_cluster_id", "day_of_week"], how='left')
    trainIn = pd.merge(trainIn, stat10, on = ["n1000mt_cluster_id", "day_of_week"], how='left')
    
    trainIn = pd.merge(trainIn, stat11, on = ["n200mt_cluster_id", "genre_name"], how='left')
    trainIn = pd.merge(trainIn, stat12, on = ["n200mt_cluster_id", "genre_name"], how='left')
    trainIn = pd.merge(trainIn, stat13, on = ["n400mt_cluster_id", "genre_name"], how='left')
    trainIn = pd.merge(trainIn, stat14, on = ["n400mt_cluster_id", "genre_name"], how='left')
    trainIn = pd.merge(trainIn, stat15, on = ["n1000mt_cluster_id", "genre_name"], how='left')
    trainIn = pd.merge(trainIn, stat16, on = ["n1000mt_cluster_id", "genre_name"], how='left')
    
    ############################
    testIn = pd.merge(testIn, stat1, on = ["air_store_id", "day_of_week"], how='left')
    testIn = pd.merge(testIn, stat2, on = ["air_store_id", "day_of_week"], how='left')
    testIn = pd.merge(testIn, stat3, on = ["area_id", "day_of_week"], how='left')
    testIn = pd.merge(testIn, stat4, on = ["area_id", "day_of_week"], how='left')
    
    testIn = pd.merge(testIn, stat_6wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    testIn = pd.merge(testIn, stat_7wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    testIn = pd.merge(testIn, stat_8wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    testIn = pd.merge(testIn, stat_9wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    testIn = pd.merge(testIn, stat_10wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')

    testIn = pd.merge(testIn, stat_6wago, on = ["air_store_id", "week_seq_id"], how='left')
    testIn = pd.merge(testIn, stat_7wago, on = ["air_store_id", "week_seq_id"], how='left')
    testIn = pd.merge(testIn, stat_8wago, on = ["air_store_id", "week_seq_id"], how='left')
    testIn = pd.merge(testIn, stat_9wago, on = ["air_store_id", "week_seq_id"], how='left')
    testIn = pd.merge(testIn, stat_10wago, on = ["air_store_id", "week_seq_id"], how='left')
    
    testIn = pd.merge(testIn, stat5, on = ["n200mt_cluster_id", "day_of_week"], how='left')
    testIn = pd.merge(testIn, stat6, on = ["n200mt_cluster_id", "day_of_week"], how='left')
    testIn = pd.merge(testIn, stat7, on = ["n400mt_cluster_id", "day_of_week"], how='left')
    testIn = pd.merge(testIn, stat8, on = ["n400mt_cluster_id", "day_of_week"], how='left')
    testIn = pd.merge(testIn, stat9, on = ["n1000mt_cluster_id", "day_of_week"], how='left')
    testIn = pd.merge(testIn, stat10, on = ["n1000mt_cluster_id", "day_of_week"], how='left')
    
    testIn = pd.merge(testIn, stat11, on = ["n200mt_cluster_id", "genre_name"], how='left')
    testIn = pd.merge(testIn, stat12, on = ["n200mt_cluster_id", "genre_name"], how='left')
    testIn = pd.merge(testIn, stat13, on = ["n400mt_cluster_id", "genre_name"], how='left')
    testIn = pd.merge(testIn, stat14, on = ["n400mt_cluster_id", "genre_name"], how='left')
    testIn = pd.merge(testIn, stat15, on = ["n1000mt_cluster_id", "genre_name"], how='left')
    testIn = pd.merge(testIn, stat16, on = ["n1000mt_cluster_id", "genre_name"], how='left')
 
    return (trainIn, testIn)

In [32]:
col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors','prefecture','city']]

In [33]:
params = {}
params['objective'] = 'reg:linear'
params['booster'] = 'gbtree'
params['eval_metric'] = 'rmse'
#params['eta'] = 0.1
params['max_depth'] = 4
params['silent'] = 1
params['subsample'] = 0.8
params['colsample_bytree'] = 0.8
params['tree_method'] = "exact"

#watchlist = [(d_train, 'train'), (d_valid, 'valid')]

In [34]:
#X = train[train['visit_date'] < dt.date(2017, 2, 21)]
#X = train[col].copy()
y = np.log1p(train['visitors']).copy()

In [35]:
X = train.copy()

In [36]:
test = test[col]

In [37]:
K = 10
kf = model_selection.KFold(n_splits = K, shuffle = True)

In [38]:
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

In [41]:
y_test_pred = 0
#K-Fold Validation for xgboost
for i, (train_index, test_index) in enumerate(kf.split(X)):
    train, val = prepareData(X.iloc[train_index, :].copy(), X.iloc[test_index, :].copy())
    # Create data for this fold
    #y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
    #X_train, X_valid = .iloc[train_index, :].copy(), X.iloc[test_index, :].copy()
    
    X_train, y_train = train[col], np.log1p(train['visitors'])
    X_valid, y_valid = val[col], np.log1p(val['visitors'])
    print("\nFold ", i)
    
    d_train = xgb.DMatrix(X_train[col],y_train)
    d_valid = xgb.DMatrix(X_valid[col], y_valid)
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]

    model = (xgb.train(params,d_train,num_boost_round=5000,evals=watchlist,
                   early_stopping_rounds=1000,verbose_eval=100))
    pred = model.predict(xgb.DMatrix(X_valid))
    print('RMSLE XGB Regressor, validation set, fold ', i, ': ', RMSLE(y_valid, pred))

    pred = model.predict(xgb.DMatrix(test[col]))
    print('Prediction length on test set, XGB Regressor, fold ', i, ': ', len(pred))
    y_test_pred += pred

    del X_train, X_valid, y_train, y_valid


Fold  0
[0]	train-rmse:1.75378	valid-rmse:1.74924
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 1000 rounds.
[100]	train-rmse:0.507492	valid-rmse:0.526114
[200]	train-rmse:0.4949	valid-rmse:0.523447
[300]	train-rmse:0.485461	valid-rmse:0.522683
[400]	train-rmse:0.476658	valid-rmse:0.522776
[500]	train-rmse:0.468939	valid-rmse:0.523061
[600]	train-rmse:0.461676	valid-rmse:0.523687
[700]	train-rmse:0.455041	valid-rmse:0.524862
[800]	train-rmse:0.448543	valid-rmse:0.525064
[900]	train-rmse:0.442195	valid-rmse:0.525733
[1000]	train-rmse:0.436329	valid-rmse:0.526236
[1100]	train-rmse:0.430443	valid-rmse:0.527303
[1200]	train-rmse:0.424942	valid-rmse:0.528283
[1300]	train-rmse:0.41955	valid-rmse:0.528788
Stopping. Best iteration:
[382]	train-rmse:0.478159	valid-rmse:0.52255

RMSLE XGB Regressor, validation set, fold  0 :  0.5291542570499939
Prediction length on test set, XGB Regressor, fold  0 :  82302



[200]	train-rmse:0.495656	valid-rmse:0.519684
[300]	train-rmse:0.485585	valid-rmse:0.519499
[400]	train-rmse:0.477277	valid-rmse:0.520176
[500]	train-rmse:0.469191	valid-rmse:0.52117
[600]	train-rmse:0.461612	valid-rmse:0.522096
[700]	train-rmse:0.454766	valid-rmse:0.522695
[800]	train-rmse:0.448711	valid-rmse:0.523204
[900]	train-rmse:0.442682	valid-rmse:0.524105
[1000]	train-rmse:0.436742	valid-rmse:0.525413
[1100]	train-rmse:0.431146	valid-rmse:0.525927
[1200]	train-rmse:0.425563	valid-rmse:0.526562
[1300]	train-rmse:0.420269	valid-rmse:0.526665
Stopping. Best iteration:
[302]	train-rmse:0.485358	valid-rmse:0.519395

RMSLE XGB Regressor, validation set, fold  8 :  0.5267762266391837
Prediction length on test set, XGB Regressor, fold  8 :  82302

Fold  9
[0]	train-rmse:1.754	valid-rmse:1.74987
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 1000 rounds.
[100]	train-rmse:0.508642	valid-rmse:0.523802


In [42]:
y_test_pred /= (K)

In [ ]:
print('RMSLE XGB Regressor, full validtion, fold  ' + str(RMSLE(np.log1p(test['visitors']).values, y_test_pred)))